In [1]:
# Import Tools

import requests
from bs4 import BeautifulSoup
import pandas as pd
import random

In [2]:
tag_urls = {
    'Entrepreneurship': 'https://medium.com/tag/entrepreneurship/archive/{0}/{1:02d}/{2:02d}'
}

In [3]:
year = 2022
selected_days = [i for i in range(1, 48)] #Every day of the year

In [4]:
def convert_day(day):
    # if it is a leap year use month_days = [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    month_days = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    m = 0
    d = 0
    while day > 0:
        d = day
        day -= month_days[m]
        m += 1
    return (m, d)

In [5]:
def get_claps(claps_str):
    if (claps_str is None) or (claps_str == '') or (claps_str.split is None):
        return 0
    split = claps_str.split('K')
    claps = float(split[0])
    claps = int(claps*1000) if len(split) == 2 else int(claps)
    return claps

In [6]:
articles_data = []
article_id = 0
n = len(selected_days)
for d in selected_days:
    month, day = convert_day(d)
    date = '{0}-{1:02d}-{2:02d}'.format(year, month, day) 
    print(f'{date}')
    for tag, url in tag_urls.items(): 
        response = requests.get(url.format(year, month, day), allow_redirects=True)
        if not response.url.startswith(url.format(year, month, day)):
            continue
        page = response.content
        soup = BeautifulSoup(page, 'html.parser')
        articles = soup.find_all(
            "div",
            class_="postArticle postArticle--short js-postArticle js-trackPostPresentation js-trackPostScrolls")
        
        for article in articles:
            
            title = article.find("h3", class_="graf--title")
            if title is None:
                continue
            title = title.contents[0]
            
            author = article.find_all("a")[0]['href'].split('?')[0].split('@')[1]
            author_url = article.find_all("a")[0]['href'].split('?')[0]
            
            subtitle = article.find("h4", class_="graf--subtitle")
            subtitle = subtitle.contents[0] if subtitle is not None else ''
            
            article_url = article.find_all("a")[3]['href'].split('?')[0]
            
            reading_time = article.find("span", class_="readingTime")
            reading_time = 0 if reading_time is None else int(reading_time['title'].split(' ')[0])
            
            responses = article.find_all("a")
            if len(responses) == 7:
                responses = responses[6].contents[0].split(' ')
                if len(responses) == 0:
                    responses = 0
                else:
                    responses = responses[0]
            else:
                responses = 0
                
            articles_data.append([article_url, title,
                         author, author_url,
                         subtitle, responses,
                         reading_time, tag, date])

2022-01-01
2022-01-02
2022-01-03
2022-01-04
2022-01-05
2022-01-06
2022-01-07
2022-01-08
2022-01-09
2022-01-10
2022-01-11
2022-01-12
2022-01-13
2022-01-14
2022-01-15
2022-01-16
2022-01-17
2022-01-18
2022-01-19
2022-01-20
2022-01-21
2022-01-22
2022-01-23
2022-01-24
2022-01-25
2022-01-26
2022-01-27
2022-01-28
2022-01-29
2022-01-30
2022-01-31
2022-02-01
2022-02-02
2022-02-03
2022-02-04
2022-02-05
2022-02-06
2022-02-07
2022-02-08
2022-02-09
2022-02-10
2022-02-11
2022-02-12
2022-02-13
2022-02-14
2022-02-15
2022-02-16


In [7]:
# Transform article data into panda dataframe.
medium_df = pd.DataFrame(articles_data, columns=[
    'url', 'title', 'author', 'author_page',
    'subtitle', 'responses', 'reading_time',
    'tag', 'date'])

In [8]:
medium_df.shape

(5073, 9)

In [9]:
medium_df = medium_df.drop_duplicates(subset=['url', 'title'], keep='first')

In [10]:
medium_df.shape

(5073, 9)

In [11]:
medium_df

,url,title,author,author_page,subtitle,responses,reading_time,tag,date
0,https://medium.com/swlh/seven-kickass-beliefs-...,Seven Kickass Beliefs About Money to Upgrade Y...,timdenning,https://medium.com/@timdenning,A job makes you an actor so…,15,6,Entrepreneurship,2022-01-01
1,https://medium.com/@aina.fadina/how-this-misfi...,How this Misfit Made it into the Venture Capit...,aina.fadina,https://medium.com/@aina.fadina,,0,4,Entrepreneurship,2022-01-01
2,https://medium.com/the-side-hustle-club/7-side...,7 Side Hustles Nobody Ever Talks About,millennialnextdoor,https://medium.com/@millennialnextdoor,,13,5,Entrepreneurship,2022-01-01
3,https://medium.com/big-on-development/how-to-a...,How to Address Tech Debt vs. Product Debt vs. ...,julius.uy,https://medium.com/@julius.uy,,0,6,Entrepreneurship,2022-01-01
4,https://richculturemedia.com/why-you-should-th...,Why You Should Throw All Your 2022 Plans in th...,davidol,https://richculturemedia.com/@davidol,When was the last time a year went according…,3,5,Entrepreneurship,2022-01-01
...,...,...,...,...,...,...,...,...,...
5068,https://medium.com/@insightssuccess.dm/themagi...,THEMAGIC5: Elevating the Experience of Swimmin...,insightssuccess.dm,https://medium.com/@insightssuccess.dm,,0,2,Entrepreneurship,2022-02-16
5069,https://medium.com/@debjanipore7/how-to-become...,How to Become Rich — Make Money Work For You,debjanipore7,https://medium.com/@debjanipore7,,0,3,Entrepreneurship,2022-02-16
5070,https://medium.com/@ochiaibunshiro/why-is-it-s...,Why is it so difficult to see phenomena as the...,ochiaibunshiro,https://medium.com/@ochiaibunshiro,,0,6,Entrepreneurship,2022-02-16
5071,https://medium.com/@team1-biznovator/register-...,Register for the BIZNOVATOR 2 week Camp for yo...,team1-biznovator,https://medium.com/@team1-biznovator,,0,1,Entrepreneurship,2022-02-16


In [12]:
medium_df.to_csv('medium-data-entrepreneurship.csv', index=True)